In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import pandas as pd
import os

## Retreive All Town Names From Website

In [16]:
#Intialize, and set Firefox options
options = webdriver.FirefoxOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--private')
#options.add_argument('--headless')

#Intialize The Firefox webdriver, and pass in the options
driver = webdriver.Firefox(options)

#Maximize the browser window
driver.maximize_window()

#Have the driver go to this website
driver.get('https://cteco.uconn.edu/projects/fish/viewer/index.html')

#Click the down arrow in the town
driver.find_element('xpath',"//input[contains(@value,'▼ ')]").click()

#Convert the page source code to BeautifulSoup object
page_source = driver.page_source
soup = BeautifulSoup(page_source,'lxml')

#Find, and grab all the town names that exist on the website
towns = []
html_towns = soup.find_all('div',class_='dijitReset dijitMenuItem')
for town in html_towns:
    towns.append(town.text)
    
#Add a delay so to verify everything will run before quitting the driver
time.sleep(10)

#Quit the selenium driver
driver.quit()

## Search and Download All Data on the Town Level

In [28]:
options = webdriver.FirefoxOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--private')
#options.add_argument('--headless') #Set this to be headless if you want
driver = webdriver.Firefox(options)
driver.maximize_window()


#Search on the town level for every town found in the code above
for town in towns:
    
    #Go to the website
    driver.get('https://cteco.uconn.edu/projects/fish/viewer/index.html')
    time.sleep(10)
    
    #Wait for search bar element to be present to find it
    search_bar = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//input[contains(@id,'cboTownSel')]"))
    )
    
    #Clear search bar
    search_bar.clear()
    
    #Enter the town in the search bar
    search_bar.send_keys(town + Keys.ENTER)
    time.sleep(10)
    
    #Wait for the apply button to be present to find it
    apply_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='btnSearch']"))
    )
    
    #Click the apply button
    apply_button.click()
    time.sleep(10)
    
    #Wait for the download results button to be present to click it
    download_results = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//a[@download='townfishresults.csv']"))
        )
    
    #Click the download results button
    download_results.click()
    
    #Located the recently downloaded file
    download_dir = r'C:\Users\12034\Downloads'
    downloaded_file_path = os.path.join(download_dir, 'townfishresults.csv')
    
    #Rename the file to the town its from, and move the file to the project dir
    time.sleep(3)
    os.rename(downloaded_file_path,os.path.join(os.getcwd(),str(town+'.csv')))
    time.sleep(3)
    try:
        os.remove(downloaded_file_path)
    except:
        pass
    
#Quit the selenium driver    
driver.quit()